<a href="https://colab.research.google.com/github/mawhy/OpenCV/blob/master/Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Processing CookBook
## Image Segmentation


In [0]:
!git clone https://github.com/PacktPublishing/Python-Image-Processing-Cookbook.git
%cp -av "/content/Python-Image-Processing-Cookbook/Chapter 06/images/" "/content/"
%cp -av "/content/Python-Image-Processing-Cookbook/Chapter 06/models/" "/content/"
%rm -rf "/content/Python-Image-Processing-Cookbook"

In [0]:
!pip install mahotas

### Segmentation by Thresholding with Otsu and Riddler-Calvard

In [0]:
%matplotlib inline
import mahotas as mh
import numpy as np
import matplotlib.pylab as plt
image = mh.imread('images/netaji.png')
thresh_otsu, thresh_rc = mh.otsu(image), mh.rc(image)
print(thresh_otsu, thresh_rc)
binary_otsu, binary_rc = image > thresh_otsu, image > thresh_rc
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 15))
axes = axes.ravel()
axes[0].imshow(image, cmap=plt.cm.gray)
axes[0].set_title('Original', size=20), axes[0].axis('off')
axes[1].hist(image.ravel(), bins=256, density=True)
axes[1].set_title('Histogram', size=20)
axes[1].axvline(thresh_otsu, label='otsu', color='green', lw=3), axes[1].axvline(thresh_rc, label='rc', color='red', lw=2)
axes[1].legend(loc='upper left', prop={'size': 20}), axes[1].grid()
axes[2].imshow(binary_otsu, cmap=plt.cm.gray)
axes[2].set_title('Thresholded (Otsu)', size=20), axes[2].axis('off')
axes[3].imshow(binary_rc, cmap=plt.cm.gray)
axes[3].set_title('Thresholded (Riddler-Calvard)', size=20), axes[3].axis('off')
plt.tight_layout()
plt.show()

In [0]:
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.filters import try_all_threshold
image = rgb2gray(imread('images/board.png'))
plt.rcParams.update({'axes.titlesize':20})
fig, ax = try_all_threshold(image, figsize=(20, 60), verbose=False)
fig.subplots_adjust(0,0,1,0.95,0.01,0.06)
plt.show()

### Skin Color Detection and Segmentation with Gaussian Mixture Model

In [0]:
# https://archive.ics.uci.edu/ml/datasets/skin+segmentation

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import pandas as pd
import seaborn as sns

df = pd.read_csv('images/Skin_NonSkin.txt', header=None, delim_whitespace=True)
df.columns = ['B', 'G', 'R', 'skin']
df.head()

In [0]:
g = sns.factorplot(data=pd.melt(df, id_vars='skin'), x='variable', y='value', \
                   hue='variable', col='skin', kind='box', palette=sns.color_palette("hls", 3)[::-1])
plt.show()

#Y = .299*r + .587*g + .114*b
df['Cb'] = np.round(128 -.168736*df.R -.331364*df.G + .5*df.B).astype(int)
df['Cr'] = np.round(128 +.5*df.R - .418688*df.G - .081312*df.B).astype(int)

df.drop(['B','G','R'], axis=1, inplace=True)
df.head()

In [0]:
df.groupby('skin').count()

In [0]:
g = sns.factorplot(data=pd.melt(df, id_vars='skin'), x='variable', y='value', \
                   hue='variable', col='skin', kind='box')
plt.show()

In [0]:
skin_data = df[df.skin==1].drop(['skin'], axis=1).to_numpy()
not_skin_data = df[df.skin==2].drop(['skin'], axis=1).to_numpy()
skin_gmm = GaussianMixture(n_components=4, covariance_type='full').fit(skin_data)
not_skin_gmm = GaussianMixture(n_components=4, covariance_type='full').fit(not_skin_data)

In [0]:
skin_gmm.means_, skin_gmm.covariances_

In [0]:
not_skin_gmm.means_, not_skin_gmm.covariances_

In [0]:
colors = ['navy', 'turquoise', 'darkorange', 'gold']

def draw_ellipses(gmm, ax):
    for n, color in enumerate(colors):
        covariances = gmm.covariances_[n][:2, :2]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1],
                                  180 + angle, color=color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        ax.set_aspect('equal', 'datalim')

def plot_gmm(gmm, data, i, title):
    h = plt.subplot(1, 2, i)
    draw_ellipses(gmm, h)
    plt.scatter(data[:, 0], data[:, 1], s=0.8, color='k')
    plt.title(title)

plt.figure(figsize=(15,8))
plot_gmm(skin_gmm, skin_data, 1, 'Skin GMM')
plot_gmm(not_skin_gmm, not_skin_data, 2, 'Not Skin GMM')
plt.show()    

In [0]:
from skimage.io import imread
from skimage.color import rgb2ycbcr, gray2rgb
image = imread('images/skin.png')
proc_image = np.reshape(rgb2ycbcr(image), (-1, 3))
skin_score = skin_gmm.score_samples(proc_image[...,1:])
not_skin_score = not_skin_gmm.score_samples(proc_image[...,1:])
result = skin_score > not_skin_score
result = result.reshape(image.shape[0], image.shape[1])
result = np.bitwise_and(gray2rgb(255*result.astype(np.uint8)), image)
plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(image), plt.axis('off'), plt.title('Original', size=20)
plt.subplot(122), plt.imshow(result), plt.axis('off'), plt.title('Skin Detected and Segmented with GMM', size=20)
plt.show()

### Medical Image Segmentation with GMM-EM

In [0]:
!git clone https://github.com/curiale/Medical-Image-Analysis-IPython-Tutorials.git

In [0]:
%cp -av "/content/Medical-Image-Analysis-IPython-Tutorials/tutorial_3/data/atlas_slicez90.nii.gz" "/content/images" 

In [0]:
!pip install SimpleITK

In [0]:
#https://github.com/curiale/Medical-Image-Analysis-IPython-Tutorials
import SimpleITK as sitk
import numpy as np
import matplotlib.pylab as plt
#import matplotlib.mlab as mlab
import scipy.stats as scs
from sklearn.mixture import GaussianMixture

# Atlas image (high-quality image - obtained by registering and averaging inter-patient T1 brain images)

# Read the brain slice image
max_int_val = 512;
image = sitk.ReadImage("images/atlas_slicez90.nii.gz", sitk.sitkFloat32)
image = sitk.RescaleIntensity(image,0.0,max_int_val)
image_data = sitk.GetArrayFromImage(image);

# Compute the parameters for the Gaussian Mixture model
np.random.seed(1)
g = GaussianMixture(n_components=4, covariance_type='diag', tol=0.01, max_iter=100, 
                n_init=1, init_params='kmeans')

# Estimate model parameters with the expectation-maximization algorithm.
g.fit(image_data[0].flatten().reshape(-1, 1)) 

# pdf of each gaussian model
def plot_pdf_models(x, g):
    we = g.weights_
    mu = g.means_
    si = np.sqrt(g.covariances_)
    for ind in range(0,we.shape[0]): 
        plt.plot(x,we[ind]*scs.norm.pdf(x, mu[ind], si[ind]),linewidth=4)

# Class probability distribution function
x = np.linspace(0,max_int_val,500)
plt.figure(figsize=(16, 5), dpi=100)
plot_pdf_models(x,g)
plt.hist(image_data.flatten(), bins=int(max_int_val/6), range=(0, max_int_val), density=True)
plt.title('Class specific probability distribution functions',fontsize=20)
plt.show()

# Single components (class posterioe)
plt.figure(figsize=(16, 3), dpi=100)
print(x.shape, g.predict_proba(x.reshape(-1,1)).shape)
plt.plot(x,g.predict_proba(x.reshape(-1,1)), linewidth=4)
plt.title('Class posterior probability under each Gaussian in the model',fontsize=20)
plt.show()

# Compute label image
label_data = g.predict(image_data[0].flatten().reshape(-1, 1)) #.flatten()) 
label_data = label_data.reshape(image_data[0].shape)
plt.figure(figsize=(15,15))
plt.subplot(121), plt.imshow(image_data[0], cmap='gray'), plt.axis('off'), plt.title('Original image', size=20)
plt.subplot(122), plt.imshow(label_data, cmap='jet'), plt.axis('off'), plt.title('Segmented image', size=20)
plt.show()

### Tumor Segmentation with Deep Learning

In [0]:
# https://github.com/polo8214/Brain-tumor-segmentation-using-deep-learning/
# https://drive.google.com/file/d/1hE9It0ZOOeIuSFvt6GdiR_0cq9inWdTy/view
# weights-full-best.h5
import keras
print(keras.__version__)
from keras.models import model_from_json
from keras.utils.vis_utils import plot_model
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray, gray2rgb
import numpy as np
import matplotlib.pylab as plt
import matplotlib.patches as mpatches
# pip3 install pydot-ng
import pydot_ng as pydot
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot #import pydotplus as pydot

loaded_model_json = open('models/model.json', 'r').read()
model = model_from_json(loaded_model_json)
model.load_weights('models/weights-full-best.h5')
plot_model(model, to_file='images/model_plot.png', show_shapes=True, show_layer_names=True)

#using Flair and T2 as input for full tumor segmentation
x = np.zeros((1,2,240,240),np.float32)
Flair = resize((rgb2gray(imread('images/Flair.png')).astype('float32')), (240,240))
T2 = resize((rgb2gray(imread('images/T2.png'))).astype('float32'), (240,240))
ground_truth = resize(rgb2gray(imread('images/ground_truth.png')), (240,240))

T2 = (T2-T2.mean()) / T2.std()
Flair = (Flair-Flair.mean()) / Flair.std()

x[:,:1,:,:] = np.reshape(Flair, (1,1,240,240))
x[:,1:,:,:] = np.reshape(T2, (1,1,240,240))

pred_full = model.predict(x)
pred_full = np.reshape(pred_full, (240,240))
tumor_overlay = 0.2*gray2rgb(T2) + 0.4*np.array([0.25,1,0.25])*gray2rgb(pred_full) + 0.4*np.array([1,0.25,0.25])*gray2rgb(ground_truth)

plt.figure(figsize=(19,20))
plt.gray()
plt.subplots_adjust(0,0,1,0.95,0.01,0.05)
plt.subplot(221), plt.title('MR T2', size=25), plt.axis('off'), plt.imshow(T2)
plt.subplot(222), plt.title('MR Flair', size=25), plt.axis('off'), plt.imshow(Flair)
plt.subplot(223), plt.title('MR Tumor Ground Truth', size=25), plt.axis('off'), plt.imshow(ground_truth)
plt.subplot(224), plt.title('Tumor Prediction', size=25), plt.axis('off'), plt.imshow(tumor_overlay)
colors, labels = ['#A76161', '#FFFFB2'], ['GT', 'Pred']
patches = [ mpatches.Patch(color=colors[i], label=labels[i]) for i in range(len(colors)) ]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0, prop={'size':25}, frameon=True, shadow=True)
plt.show()

### Watershed Segmentation

In [0]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pylab as plt
img = sitk.ReadImage('images/fib_sem_bacillus_subtilis_slice_118.png', sitk.sitkFloat32)
f = sitk.RescaleIntensityImageFilter()
img = f.Execute(img, 0, 255)
plt.figure(figsize=(20,26))
plt.gray()
plt.subplots_adjust(0,0,1,0.95,0.05,0.05), plt.axis('off')
plt.subplot(421), plt.imshow(sitk.GetArrayFromImage(img)), plt.title('Original image', size=20)
plt.subplot(422), plt.hist(sitk.GetArrayViewFromImage(img).flatten(), bins=100), plt.title('histogram of pixel values', size=20)
thresh_value = 120
thresh_img = img>thresh_value
plt.subplot(423), plt.imshow(sitk.GetArrayFromImage(sitk.LabelOverlay(img, thresh_img))/255), plt.title('Binary Segmentation', size=20), plt.axis('off')
cleaned_thresh_img = sitk.BinaryOpeningByReconstruction(thresh_img, [10, 10, 10])
cleaned_thresh_img = sitk.BinaryClosingByReconstruction(cleaned_thresh_img, [10, 10, 10])
#from collections import Counter
#print(Counter(sitk.GetArrayFromImage(cleaned_thresh_img).flatten()))
plt.subplot(424), plt.imshow(sitk.GetArrayFromImage(cleaned_thresh_img)), plt.title('Cleaned Binary Segmentation', size=20), plt.axis('off')
dist_img = sitk.SignedMaurerDistanceMap(cleaned_thresh_img != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False)
radius = 10
# Seeds have a distance of "radius" or more to the object boundary, they are uniquely labelled.
seeds = sitk.ConnectedComponent(dist_img < -radius)
# Relabel the seed objects using consecutive object labels while removing all objects with less than 15 pixels.
seeds = sitk.RelabelComponent(seeds, minimumObjectSize=15)
# Run the watershed segmentation using the distance map and seeds.
ws = sitk.MorphologicalWatershedFromMarkers(dist_img, seeds, markWatershedLine=True)
ws = sitk.Mask( ws, sitk.Cast(cleaned_thresh_img, ws.GetPixelID()))
img = sitk.Cast(img, sitk.sitkUInt8)
#print(img.GetPixelIDTypeAsString(), seeds.GetPixelIDTypeAsString())
plt.subplot(425), plt.imshow(sitk.GetArrayFromImage(dist_img)), plt.title('Segmentation Distance', size=20), plt.axis('off')
plt.subplot(426), plt.imshow(sitk.GetArrayFromImage(sitk.LabelOverlay(img, seeds))), plt.title('Watershed Seeds', size=20), plt.axis('off')
plt.subplot(427), plt.imshow(sitk.GetArrayFromImage(sitk.LabelOverlay(img, ws))), plt.title('Binary Watershed Labeling', size=20), plt.axis('off')
plt.show()

### Image segmentation with Self Organizing Maps (SOM)

In [0]:
!pip install minisom


In [0]:
from minisom import MiniSom

import numpy as np
import matplotlib.pyplot as plt

def segment_with_SOM(image, nx, ny, sigma=1., n=500):
    pixels = np.reshape(image, (image.shape[0]*image.shape[1], 3))
    # SOM initialization and training
    print('training...')
    som = MiniSom(x=nx, y=ny, input_len=3, sigma=sigma, learning_rate=0.2)  # nxxny final colors
    som.random_weights_init(pixels)
    starting_weights = som.get_weights().copy()  # saving the starting weights
    som.train_random(pixels, n)
    print('quantization...')
    qnt = som.quantization(pixels)  # quantize each pixels of the image
    print('building new image...')
    clustered = np.zeros(image.shape)
    for i, q in enumerate(qnt):  # place the quantized values into a new image
        clustered[np.unravel_index(i, dims=(image.shape[0], image.shape[1]))] = q
    print('done.')
    final_weights = som.get_weights()
    return clustered, starting_weights, final_weights

from skimage import img_as_float
image = img_as_float(plt.imread('images/apples.png'))

clustered, starting_weights, final_weights = segment_with_SOM(image, 1, 2, .1)
colors = np.unique(clustered.reshape(-1,3), axis=0)
clustered_binary = np.zeros_like(clustered)
clustered_binary[np.where((clustered[...,0]==colors[1][0]) & \
                 (clustered[...,1]==colors[1][1]) & \
                 (clustered[...,2]==colors[1][2]))] = 1.

plt.figure(figsize=(20, 15))
plt.subplot(321), plt.title('original', size=20), plt.axis('off'), plt.imshow(image)
plt.subplot(322), plt.title('segmented', size=20), plt.axis('off'), plt.imshow(clustered)
plt.subplot(323), plt.title('initial colors', size=20), plt.axis('off'), plt.imshow(starting_weights, interpolation='none')
plt.subplot(324), plt.title('learned colors', size=20), plt.axis('off'), plt.imshow(final_weights, interpolation='none')
plt.subplot(325), plt.title('segmented (binary)', size=20), plt.axis('off'), plt.imshow(clustered_binary)
plt.tight_layout()
plt.show()

clustered, starting_weights, final_weights = segment_with_SOM(image, 5, 5)
plt.figure(figsize=(20, 10))
plt.subplot(221), plt.title('original', size=20), plt.axis('off'), plt.imshow(image)
plt.subplot(222), plt.title('segmented (color-quantized)', size=20), plt.axis('off'), plt.imshow(clustered)
plt.subplot(223), plt.title('initial colors', size=20), plt.axis('off'), plt.imshow(starting_weights, interpolation='none')
plt.subplot(224), plt.title('learned colors', size=20), plt.axis('off'), plt.imshow(final_weights, interpolation='none')
plt.tight_layout()
plt.show()


### Clustering digits images with SOM

In [0]:
from minisom import MiniSom
import numpy as np
import matplotlib.pyplot as plt
from pylab import pcolor
from collections import defaultdict
from sklearn import datasets
from sklearn.preprocessing import scale

digits = datasets.load_digits(n_class=10)
data = digits.data  # matrix where each row is a vector that represent a digit.
data = scale(data)
num = digits.target  # num[i] is the digit represented by data[i]

som = MiniSom(30, 30, 64, sigma=4, learning_rate=0.5, neighborhood_function='triangle')
som.pca_weights_init(data)
print("Training...")
som.train_random(data, 5000)  # random training
print("\n...ready!")

In [0]:
plt.figure(figsize=(15, 12))
pcolor(som.distance_map().T, cmap='coolwarm')
plt.colorbar()
wmap = defaultdict(list)
im = 0
for x, t in zip(data, num):  # scatterplot
    w = som.winner(x)
    wmap[w].append(im)
    plt. text(w[0]+.5,  w[1]+.5,  str(t),
              color=plt.cm.Dark2(t / 10.), fontdict={'weight': 'bold',  'size': 11})
    im = im + 1
plt.axis([0, som.get_weights().shape[0], 0,  som.get_weights().shape[1]])
plt.show()

In [0]:
print(wmap[23,15])
# [581, 598, 1361] # 3 digits assigned to node (23,15)
plt.gray()
for index in wmap[23,15]:
    plt.figure(figsize=(1,1))
    plt.imshow(np.reshape(digits.images[index], (8,-1)))
    plt.title(digits.target[index])
    plt.axis('off')
    plt.show()

### Deep Image Instance Segmentation

In [0]:
# https://www.learnopencv.com/deep-learning-based-object-detection-and-instance-segmentation-using-mask-r-cnn-in-opencv-python-c/

In [0]:
!wget http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
!tar -zxvf mask_rcnn_inception_v2_coco_2018_01_28.tar.gz



In [0]:
%cp "/content/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb" "/content/models"

In [0]:
# import the necessary packages
import numpy as np
import colorsys
import time
import cv2
import os
import random
import matplotlib.pylab as plt

print(cv2.__version__)
# 4.1.0

def random_colors(N, bright=True):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return 256*np.array(colors)


model_path = 'models'
conf = 0.5
thresh = 0.3

# load the COCO class labels our Mask R-CNN was trained on
labels_path = os.path.sep.join([model_path, "object_detection_classes_coco.txt"])
labels = open(labels_path).read().strip().split("\n")

# derive the paths to the Mask R-CNN weights and model configuration
weights_path = os.path.sep.join([model_path, "frozen_inference_graph.pb"])
config_path = os.path.sep.join([model_path, "mask_rcnn_inception_v2_coco_2018_01_28.pbtxt"])

print(weights_path, config_path)

# load our Mask R-CNN trained on the COCO dataset (90 classes)
# from disk
print("[INFO] loading Mask R-CNN from disk...")
net = cv2.dnn.readNetFromTensorflow(weights_path, config_path)

image = cv2.imread('images/pets.png')
original = image.copy()

blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
net.setInput(blob)
(boxes, masks) = net.forward(["detection_out_final",  "detection_masks"])

num_classes = masks.shape[1]
num_detections = boxes.shape[2]
print('# instances: {}'.format(num_detections))
colors = random_colors(num_detections)
print("# classes: {}".format(num_classes))


h = image.shape[0]
w = image.shape[1]

for i in range(num_detections):
    
    box = boxes[0, 0, i]
    mask = masks[i]
    score = box[2]
    
    if score > conf:
        
        class_id = int(box[1])
        print(class_id, score)

        left = int(w * box[3])
        top = int(h * box[4])
        right = int(w * box[5])
        bottom = int(h * box[6])

        left = max(0, min(left, w - 1))
        top = max(0, min(top, h - 1))
        right = max(0, min(right, w - 1))
        bottom = max(0, min(bottom, h - 1))

        # Extract the mask for the object
        class_mask = mask[class_id]

        # colorize and show the mask on the image
        label = labels[class_id]
    
        # Resize the mask, threshold, color and apply it on the image
        class_mask = cv2.resize(class_mask, (right - left + 1, bottom - top + 1))
        mask = (class_mask > thresh)
        roi = image[top:bottom+1, left:right+1][mask]

        color_index = np.random.randint(0, len(colors)-1)
        color = np.array(colors[color_index])

        image[top:bottom+1, left:right+1][mask] = (0.4*color + 0.6 * roi).astype(np.uint8)

        # Draw the contours on the image
        mask = mask.astype(np.uint8)
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(image[top:bottom+1, left:right+1], contours, -1, color, 3, cv2.LINE_8, hierarchy, 100)
        
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        top = max(top, label_size[1])
        cv2.putText(image, label, ((left + right)//2, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)


cv2.imwrite('images/instance_seg_out.png', image)

In [0]:
plt.figure(figsize=(20,8))
plt.subplot(121), plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Original', size=20)
plt.subplot(122), plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), plt.axis('off'), plt.title('Instance segmentation (Mask_RCNN)', size=20)
plt.tight_layout()
plt.show()

### Semantic Segmentation with DeepLab

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
pip install tfgraphviz

In [0]:
!wget http://download.tensorflow.org/models/deeplabv3_pascal_trainval_2018_01_04.tar.gz

In [0]:
!tar -zxvf deeplabv3_pascal_trainval_2018_01_04.tar.gz

In [0]:
!mkdir model2

In [0]:
%cp "/content/deeplabv3_pascal_trainval/frozen_inference_graph.pb" "/content/model2"

In [0]:
# https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/model_zoo.md
# http://download.tensorflow.org/models/deeplabv3_pascal_trainval_2018_01_04.tar.gz

from PIL import Image
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow.compat.v1 as tf
from tensorflow.io.gfile import GFile
#io.gfile
#from tensorflow.gfile import GFile
import tfgraphviz as tfg
import os

def run_semantic_segmentation(image, model_path):

    input_tensor_name = 'ImageTensor:0'
    output_tensor_name = 'SemanticPredictions:0'
    input_size = 513

    graph = tf.Graph()

    graph_def = None
    with GFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with graph.as_default():
      tf.import_graph_def(graph_def, name='')
    
    sess = tf.Session(graph=graph)

    width, height = image.size
    resize_ratio = 1.0 * input_size / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = sess.run(
        output_tensor_name,
        feed_dict={input_tensor_name: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    
    return resized_image, seg_map, graph


def create_pascal_label_colormap():
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)
  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3
  return colormap


def label_to_color_image(label):
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')
  colormap = create_pascal_label_colormap()
  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')
  return colormap[label]


def visualize_segmentation(image, seg_map):
  plt.figure(figsize=(20, 15))
  plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95, wspace=0.05, hspace=0.05)
  plt.subplot(221), plt.imshow(image), plt.axis('off'), plt.title('input image', size=20)
  plt.subplot(222)
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image), plt.axis('off'), plt.title('segmentation map', size=20)
  plt.subplot(223), plt.imshow(image), plt.imshow(seg_image, alpha=0.7), plt.axis('off'), plt.title('segmentation overlay', size=20)
  unique_labels = np.unique(seg_map)
  ax = plt.subplot(224)
  plt.imshow(full_color_map[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right(), plt.yticks(range(len(unique_labels)), label_names[unique_labels]), plt.xticks([], [])
  ax.tick_params(width=0.0, labelsize=20), plt.grid('off')
  plt.show()


label_names = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

In [0]:
full_label_map = np.arange(len(label_names)).reshape(len(label_names), 1)
full_color_map = label_to_color_image(full_label_map)
image, seg_map, graph = run_semantic_segmentation(Image.open('images/pets.png'), 'model2/frozen_inference_graph.pb')

In [0]:
tfg.board(graph)

In [0]:
visualize_segmentation(image, seg_map)

In [0]:
image, seg_map, graph = run_semantic_segmentation(Image.open('images/road.png'), 'model2/frozen_inference_graph.pb')
visualize_segmentation(image, seg_map)

### Semantic Segmentation with FCN

In [0]:
!wget http://dl.caffe.berkeleyvision.org/fcn8s-heavy-pascal.caffemodel

In [0]:
%cp "/content/fcn8s-heavy-pascal.caffemodel" "/content/models"

In [0]:
# http://dl.caffe.berkeleyvision.org/fcn8s-heavy-pascal.caffemodel
import matplotlib.pylab as plt
import numpy as np
import imutils
import time
import cv2
print(cv2.__version__)

# load the class label names
lines = open('models/pascal-classes.txt').read().strip().split("\n")
classes, colors = [], []
for line in lines:
    words = line.split(' ')
    classes.append(words[0])
    colors.append(list(map(int, words[1:]))) 
colors = np.array(colors, dtype="uint8")
#print(classes)

# initialize the legend visualization
legend = np.zeros(((len(classes) * 25) + 25, 300, 3), dtype="uint8")

# loop over the class names + colors
for (i, (className, color)) in enumerate(zip(classes, colors)):
     # draw the class name + color on the legend
     color = [int(c) for c in color]
     cv2.putText(legend, className, (5, (i * 25) + 17), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
     cv2.rectangle(legend, (100, (i * 25)), (300, (i * 25) + 25), tuple(color), -1)

# load our serialized model from disk
print("[INFO] loading model...")
model = cv2.dnn.readNetFromCaffe('models/fcn8s-heavy-pascal.prototxt',
                                 'models/fcn8s-heavy-pascal.caffemodel')

# load the input image, resize it, and construct a blob from it,
# but keeping mind mind that the original input image dimensions
start = time.time()
image = cv2.imread('images/cycling.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = imutils.resize(image, width=500)

# perform a forward pass using the segmentation model
blob = cv2.dnn.blobFromImage(image, 1, (image.shape[1],image.shape[0]))
model.setInput(blob)
output = model.forward()
end = time.time()

# show the amount of time inference took
print("[INFO] inference took {:.4f} seconds".format(end - start))

# infer the total number of classes along with the spatial dimensions
# of the mask image via the shape of the output array
(num_classes, height, width) = output.shape[1:4]

# our output class ID map will be num_classes x height x width in
# size, so we take the argmax to find the class label with the
# largest probability for each and every (x, y)-coordinate in the
# image
labels = output[0].argmax(0)

# given the class ID map, we can map each of the class IDs to its
# corresponding color
mask = colors[labels]

# resize the mask and class map such that its dimensions match the
# original size of the input image (we're not using the class map
# here for anything else but this is how you would resize it just in
# case you wanted to extract specific pixels/classes)
mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
labels = cv2.resize(labels, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# perform a weighted combination of the input image with the mask to
# form an output visualization
output = ((0.4 * image) + (0.6 * mask)).astype("uint8")
legend = imutils.resize(legend,  height=output.shape[0])

plt.figure(figsize=(20,30))
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95, wspace=0.05, hspace=0.05)
plt.subplot(221), plt.imshow(image), plt.axis('off'), plt.title('Original Image', size=30)
plt.subplot(222), plt.imshow(mask), plt.axis('off'), plt.title('Segmentation map', size=30)
plt.subplot(223), plt.imshow(output), plt.axis('off'), plt.title('Image with Segmentaion overlay', size=30)
plt.subplot(224), plt.imshow(legend), plt.axis('off'), plt.title('legends', size=30)
plt.show()

### Semantic Segmentation with XCeption Net

In [0]:
!wget http://download.tensorflow.org/models/deeplabv3_cityscapes_train_2018_02_06.tar.gz

In [0]:
!tar -zxvf deeplabv3_cityscapes_train_2018_02_06.tar.gz

In [0]:
!mkdir model3

In [0]:
%cp "/content/deeplabv3_cityscapes_train/frozen_inference_graph.pb" "/content/model3"

In [0]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pylab as plt
from matplotlib import gridspec
import tensorflow as tf
from tensorflow.io.gfile import GFile
import tensorflow.compat.v1 as tf

def label_to_color_image(label):
    
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = np.array([
        [128,  64, 128],
        [244,  35, 232],
        [ 70,  70,  70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170,  30],
        [220, 220,   0],
        [107, 142,  35],
        [152, 251, 152],
        [ 70, 130, 180],
        [220,  20,  60],
        [255,   0,   0],
        [  0,   0, 142],
        [  0,   0,  70],
        [  0,  60, 100],
        [  0,  80, 100],
        [  0,   0, 230],
        [119,  11,  32],
        [  0,   0,   0]], dtype=np.uint8)

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

def visualize_segmentation(image, seg_map):
  plt.figure(figsize=(20, 15))
  plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95, wspace=0.05, hspace=0.05)
  plt.subplot(221), plt.imshow(image), plt.axis('off'), plt.title('input image', size=20)
  plt.subplot(222)
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image), plt.axis('off'), plt.title('segmentation map', size=20)
  plt.subplot(223), plt.imshow(image), plt.imshow(seg_image, alpha=0.7), plt.axis('off'), plt.title('segmentation overlay', size=20)
  unique_labels = np.unique(seg_map)
  ax = plt.subplot(224)
  plt.imshow(full_color_map[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right(), plt.yticks(range(len(unique_labels)), label_names[unique_labels]), plt.xticks([], [])
  ax.tick_params(width=0.0, labelsize=20), plt.grid('off')
  plt.show()

label_names = np.asarray([
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck',
    'bus', 'train', 'motorcycle', 'bicycle', 'void'])

full_label_map = np.arange(len(label_names)).reshape(len(label_names), 1)
full_color_map = label_to_color_image(full_label_map)

def run_semantic_segmentation(image, model_path):
    graph = tf.Graph()
    graph_def = None
    with GFile(model_path, 'rb') as f:
        #graph_def = tf.GraphDef()
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    if graph_def is None:
        raise RuntimeError('Cannot find inference graph in tar archive.')
    with graph.as_default():
        tf.import_graph_def(graph_def, name='')
    sess = tf.Session(graph=graph)
    width, height = image.size
    target_size = (2049,1025)  # size of Cityscapes images
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = sess.run('SemanticPredictions:0',
            feed_dict={'ImageTensor:0': [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]  # expected batch size = 1
    if len(seg_map.shape) == 2:
        seg_map = np.expand_dims(seg_map,-1)  # need an extra dimension for cv.resize
    seg_map = cv2.resize(seg_map, (width,height), interpolation=cv2.INTER_NEAREST)
    return seg_map

#mobilenet: 'http://download.tensorflow.org/models/deeplabv3_mnv2_cityscapes_train_2018_02_05.tar.gz'
#xception: 'http://download.tensorflow.org/models/deeplabv3_cityscapes_train_2018_02_06.tar.gz'

model = 'model3/frozen_inference_graph.pb'
image = 'images/road.png'
image = Image.open(image)
seg_map = run_semantic_segmentation(image, model)
visualize_segmentation(image, seg_map)

### RandomWalk Segmentation with scikit-image

In [0]:
# https://www.isro.gov.in/pslv-c25-mars-orbiter-mission/pictures-mars-colour-camera-mcc-onboard-india%E2%80%99s-mars-orbiter
# https://www.isro.gov.in/image-galleries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.segmentation import random_walker
from skimage import img_as_float
from skimage.exposure import rescale_intensity
from skimage.io import imread
from skimage.color import rgb2gray

def colorbar(mappable):
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

img = imread('images/earth_by_MCC.png')
#img = rgb2gray(img)
mask = imread('images/earth_by_MCC_mask.png')
markers = np.zeros(img.shape[:2],np.uint8)
markers[(mask[...,0] >= 200)&(mask[...,1] <= 20)&(mask[...,2] <= 20)] = 1
markers[(mask[...,0] <= 20)&(mask[...,1] >= 200)&(mask[...,2] <= 20)] = 2
#print(np.unique(markers))

# Run random walker algorithm
labels = random_walker(img, markers, beta=9, mode='bf', multichannel=True)
#print(np.unique(labels))
labels2 = random_walker(img, markers, beta=9, mode='bf', multichannel=True, return_full_prob = True)
#print(labels2.shape)

# Plot results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 18), sharex=True, sharey=False)
fig.subplots_adjust(0,0,1,0.975,0.01,0.05)
ax1.imshow(mask, interpolation='nearest'), ax1.axis('off')
ax1.set_title('Original Image with Markers', size=25)
ax2.imshow(img, interpolation='nearest'), ax2.contour(labels, linewidths=5, colors='r'), ax2.axis('off')
ax2.set_title('Segmentation Contour', size=25)
ax3.imshow(labels, cmap='gray', interpolation='nearest'), ax3.axis('off')
ax3.set_title('Segmentation', size=25)
prob = ax4.imshow(labels2[1,...], cmap='inferno', interpolation='nearest')
ax4.axis('off'), ax4.set_title('Segmentation Probabilities', size=25)
colorbar(prob)
#fig.colorbar(prob, ax=ax4, orientation="horizontal", pad=0.01)
plt.show()

In [0]:
img = imread('images/CT_bones.png')
#img = rgb2gray(img)
mask = imread('images/CT_bones_mask.png')
markers = np.zeros(img.shape[:2],np.uint8)
markers[(mask[...,0] == 255)&(mask[...,1] == 0)&(mask[...,2] == 0)] = 3
markers[(mask[...,0] == 0)&(mask[...,1] == 255)&(mask[...,2] == 0)] = 2
markers[(mask[...,0] == 0)&(mask[...,1] == 0)&(mask[...,2] == 255)] = 1
#print(np.unique(markers))

# Run random walker algorithm
labels = random_walker(img, markers, beta=9, mode='bf', multichannel=True)
#print(np.unique(labels))
labels2 = random_walker(img, markers, beta=9, mode='bf', multichannel=True, return_full_prob = True)
#print(labels2.shape)
# Plot results
from skimage.color import gray2rgb
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(18, 20), sharex=True, sharey=False)
fig.subplots_adjust(0,0,1,0.975,0.01,0.05)
ax1.imshow(img, interpolation='nearest'), ax1.axis('off')
ax1.set_title('Original Image', size=25)
ax2.imshow(mask, interpolation='nearest'), ax2.axis('off')
ax2.set_title('Original Image with Markers', size=25)
ax3.imshow(img, interpolation='nearest'), ax3.contour(labels, linewidths=5, colors='r'), ax3.axis('off')
ax3.set_title('Segmentation Contour', size=25)
labels = gray2rgb(labels) #labels[...,np.newaxis]
labels[labels[...,0]==1] = [128,128,255]
labels[labels[...,0]==2] = [128,255,128]
labels[labels[...,0]==3] = [255,128,128]
ax4.imshow((0.6*labels + 0.4*img).astype(np.uint8), cmap='jet', interpolation='nearest'), ax4.axis('off')
ax4.set_title('Segmentation', size=25)
prob = ax5.imshow(labels2[2,...], cmap='jet', interpolation='nearest')
ax5.axis('off'), ax5.set_title('Segmentation Probabilities', size=25)
colorbar(prob)
prob = ax6.imshow(labels2[1,...], cmap='jet', interpolation='nearest')
ax6.axis('off'), ax6.set_title('Segmentation Probabilities', size=25)
colorbar(prob)
#fig.colorbar(prob, ax=ax4, orientation="horizontal", pad=0.01)
plt.show()